In [142]:
%pip install pfapack
%pip install npy-reader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [118]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
from pfapack import pfaffian
import math
import os

In [119]:
def create_u_matrix(N1, N2, bc1, bc2):
    N = N1 * N2 # N1 为 a_1 方向 unit cell 数量
    M = np.zeros((2 * N, 2 * N)) # M 储存 u_jk(j 属于 A 子格，k 属于 B 子格)构型
    u = np.zeros((2 * N, 2 * N))
    
    if bc1 == "PBC":
        sign1 = 1
    elif bc1 == "APBC":
        sign1 = -1
    if bc2 == "PBC":
        sign2 = 1
    elif bc2 == "APBC":
        sign2 = -1    
    
    def index(n1, n2, sub):
        return n1 + n2 * N1 + N1 * N2 * sub # A子格对应 sub=0,B子格对应 sub=1
    
    for n2 in range(N2):
        for n1 in range(N1):
            j = index(n1, n2, 0) # j 为(n1, n2)unit cell 内 A 子格格点的单指标
            
            kx = index(n1, n2, 1) # kx 为(n1, n2)unit cell 内 B 子格格点的单指标
            M[j, kx] = 1 # j 与 kx 由 x-bond 相连，一定不跨越边界
            
            ky = index((n1 + 1) % N1, n2, 1) # ky 为 (n1, n2) 往 a1 方向走一步得到的 unit cell 内 B 子格格点的单指标j 与 ky 由 y-bond 相连
            if n1 == N1 - 1:
                M[j, ky] = sign1
            else:
                M[j, ky] = 1

            kz = index(n1, (n2 + 1) % N2, 1) # kz 为 (n1, n2) 往 a2 方向走一步得到的 unit cell 内 B 子格格点的单指标，j 与 kz 由 z-bond 相连
            if n2 == N2 - 1:
                M[j, kz] = sign2
            else:
                M[j, kz] = 1
      
    u = M - M.T                        
    return u

In [120]:
def create_t_matrix(N1, N2, Kx, Ky, Kz, u):
    N = N1 * N2
    t = np.zeros((N, N))
    K = np.array([Kx, Ky, Kz])
    for i in range(N):
        for j in range(N):
            sum = 0
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            
            for delta_vec in np.array([[0, 0], [1, 0], [0, 1]]):
                tmp1 = (r_j_vec[0] - delta_vec[0] + N1) % N1
                tmp2 = (r_j_vec[1] - delta_vec[1] + N2) % N2
                if (np.array_equal(r_i_vec, np.array([tmp1, tmp2]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                tmp3 = delta_vec[0] + delta_vec[1] * 2
                K_delta = K[tmp3]

                sum += delta_func * K_delta
                
            sum *= u[i, j + N]
            t[i, j] = sum
    return t

In [121]:
def create_t_plus_matrix(N1, N2, u):
    N = N1 * N2
    t_plus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([1, 0]), np.array([0, 1]), np.array([0, 0])]
    list2 = [np.array([0, 1]), np.array([0, 0]), np.array([1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] + N1) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] + N2) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] + N1) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] + N2) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                sum += delta_func * u[index1(r_i_vec, 1), index1(np.array([tmp1, tmp2]), 0) ] * u[index1(r_j_vec, 1), index1(np.array([tmp3, tmp4]), 0)]

            t_plus[i][j] = sum
            
    return t_plus

In [122]:
def create_t_minus_matrix(N1, N2, u):
    N = N1 * N2
    t_minus = np.zeros((N, N))
    def index1(UC_vec, sub):
        return N1 * UC_vec[1] + UC_vec[0] + sub * N
    list1 = [np.array([-1, 0]), np.array([0, -1]), np.array([0, 0])]
    list2 = [np.array([0, -1]), np.array([0, 0]), np.array([-1, 0])]
    for i in range(N):
        for j in range(N):
            r_i_vec = np.array([i % N1, i // N1])
            r_j_vec = np.array([j % N1, j // N1])
            sum = 0
            
            for delta1_vec, delta2_vec in zip(list1, list2):
                tmp1 = (r_i_vec[0]-delta1_vec[0] ) % N1 # r_i - delta_1 的 a_1 分量，考虑了周期性边界条件
                tmp2 = (r_i_vec[1]-delta1_vec[1] ) % N2 # r_i - delta_1 的 a_2 分量，考虑了周期性边界条件
                tmp3 = (r_j_vec[0]-delta2_vec[0] ) % N1 # r_j - delta_2 的 a_1 分量，考虑了周期性边界条件
                tmp4 = (r_j_vec[1]-delta2_vec[1] ) % N2 # r_j - delta_2 的 a_2 分量，考虑了周期性边界条件
                if (np.array_equal(np.array([tmp1, tmp2]) , np.array([tmp3, tmp4]))):
                    delta_func = 1
                else:
                    delta_func = 0
                    
                sum += delta_func * u[index1(r_i_vec, 0), index1(np.array([tmp1, tmp2]), 1) ] * u[index1(r_j_vec, 0), index1(np.array([tmp3, tmp4]), 1)]

            t_minus[i][j] = sum
            
    return t_minus

In [123]:
def create_P_times_P_prime_matrix(N):
    P = np.zeros((2 * N, 2* N))
    for i in range(N):
        P[i, 2 * i] = 1
        P[i + N, 2 * i + 1] = 1
    # 计算 P_prime 矩阵
    I_N = np.eye(N) # N \times N 单位矩阵
    P_prime = np.kron(I_N, 0.5 * np.array([[1, 1j], [1, -1j]]))
    #print("P_prime:\n", P_prime)
    P_times_P_prime = np.dot(P, P_prime) # 矩阵乘法
    return P_times_P_prime

In [124]:
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

In [125]:
def save_positive_eigvecs(type_name, N1, N2, bc1, bc2, eigvals, eigvecs):
    N = N1 * N2
    folder = f"{type_name}_eigvecs"
    ensure_dir(folder)
    filename = f"{type_name}_eigvecs_{N1}-times-{N2}_{bc1}-{bc2}.npy"
    filepath = os.path.join(folder, filename)
    np.save(filepath, eigvecs[N:])
    print(f"[saved eigvecs] {filepath}")

In [126]:
def save_gs_energy(type_name, N1, N2, gs_energy_list):
    folder = f"{type_name}_GS-energy"
    ensure_dir(folder)

    filename = f"{type_name}_GS_energy_{N1}-times-{N2}.npy"
    filepath = os.path.join(folder, filename)
    np.save(filepath, np.array(gs_energy_list))
    print(f"[saved GS-energy] {filepath}")

In [127]:
Kx, Ky, Kz, kappa = 1, 1, 1, 0.1

In [128]:
start_size = 3
end_size = 5
for size in range(start_size, end_size+1):
    N1 = size
    N2 = size
    N = N1 * N2
    H_0_gs_energy_list = []
    H_total_gs_energy_list = []
    for bc1 in ["PBC", "APBC"]:
        for bc2 in ["PBC", "APBC"]:
            u = create_u_matrix(N1, N2, bc1, bc2)
            t = create_t_matrix(N1, N2, Kx, Ky, Kz, u)
            Xi = t + t.T
            Delta = -(t - t.T)
            H0 = np.block([
                [Xi, Delta],
                [-Delta, -Xi.T]
            ])
            eigvals_0, eigvecs_0 = np.linalg.eigh(H0)
            ground_state_energy_0 = np.sum(eigvals_0[:N]) / 2 # 计算无磁场时基态能量
            save_positive_eigvecs("H_0", N1, N2, bc1, bc2, eigvals_0, eigvecs_0)
            H_0_gs_energy_list.append(ground_state_energy_0)
            ############ 有磁场 ############
            # 计算磁场项哈密顿量
            t_plus = create_t_plus_matrix(N1, N2, u)
            t_minus = create_t_minus_matrix(N1, N2, u)
            Xi_prime = np.zeros((N, N), dtype = complex)
            Delta_prime = np.zeros((N, N), dtype = complex)         
            Xi_prime = 1j * kappa * (t_plus + t_minus - (t_plus + t_minus).T)
            Delta_prime = 1j * kappa * ((t_minus - t_plus) - (t_minus - t_plus).T)
    
            H_kappa = np.block([
                [Xi_prime, Delta_prime],
                [Delta_prime, Xi_prime]
            ])
            H_total = H0 + H_kappa # 总哈密顿量
            #print("H_total", H_total)
            eigvals_total, eigvecs_total = np.linalg.eigh(H_total)
            ground_state_energy_total = np.sum(eigvals_total[:N]) / 2 # 计算有磁场时体系基态能量
            save_positive_eigvecs("H_total", N1, N2, bc1, bc2, eigvals_total, eigvecs_total)
            H_total_gs_energy_list.append(ground_state_energy_0)
            
    save_gs_energy("H_0", N1, N2, H_0_gs_energy_list)
    save_gs_energy("H_total", N1, N2, H_total_gs_energy_list)
            

[saved eigvecs] H_0_eigvecs\H_0_eigvecs_3-times-3_PBC-PBC.npy
[saved eigvecs] H_total_eigvecs\H_total_eigvecs_3-times-3_PBC-PBC.npy
[saved eigvecs] H_0_eigvecs\H_0_eigvecs_3-times-3_PBC-APBC.npy
[saved eigvecs] H_total_eigvecs\H_total_eigvecs_3-times-3_PBC-APBC.npy
[saved eigvecs] H_0_eigvecs\H_0_eigvecs_3-times-3_APBC-PBC.npy
[saved eigvecs] H_total_eigvecs\H_total_eigvecs_3-times-3_APBC-PBC.npy
[saved eigvecs] H_0_eigvecs\H_0_eigvecs_3-times-3_APBC-APBC.npy
[saved eigvecs] H_total_eigvecs\H_total_eigvecs_3-times-3_APBC-APBC.npy
[saved GS-energy] H_0_GS-energy\H_0_GS_energy_3-times-3.npy
[saved GS-energy] H_total_GS-energy\H_total_GS_energy_3-times-3.npy
[saved eigvecs] H_0_eigvecs\H_0_eigvecs_4-times-4_PBC-PBC.npy
[saved eigvecs] H_total_eigvecs\H_total_eigvecs_4-times-4_PBC-PBC.npy
[saved eigvecs] H_0_eigvecs\H_0_eigvecs_4-times-4_PBC-APBC.npy
[saved eigvecs] H_total_eigvecs\H_total_eigvecs_4-times-4_PBC-APBC.npy
[saved eigvecs] H_0_eigvecs\H_0_eigvecs_4-times-4_APBC-PBC.npy
[saved 

In [134]:
arr = np.load("H_total_eigvecs/H_total_eigvecs_3-times-3_APBC-PBC.npy")
print(arr)


[[ 5.69845258e-02+5.92136998e-03j  4.58852416e-01+4.76802234e-02j
   4.01499756e-06+1.02163061e-05j -1.49230797e-01-3.79723145e-01j
  -2.98037946e-02-2.40471358e-02j -3.81690200e-01-2.17012483e-01j
   1.23445525e-01-1.46364067e-02j -9.33289142e-02-2.37288058e-01j
  -3.28686844e-01-5.54623286e-02j  1.02145227e-15+1.49480995e-16j
  -1.30794839e-01-5.74395400e-02j  1.48312908e-02-9.40911557e-02j
  -3.59755129e-01+8.44825617e-02j  5.52346870e-02-2.52096388e-02j
   2.00880856e-03-6.95364703e-03j  6.55071008e-02-2.26757923e-01j
  -4.27862016e-03+5.23828215e-04j -7.75389008e-02+9.49302871e-03j]
 [-3.71025206e-01+2.96068499e-03j  2.79041973e-01+2.38401117e-02j
  -3.51578052e-01+3.51631141e-02j -7.46248576e-02-1.89860627e-01j
  -5.59889315e-02+3.71915209e-02j -2.15448496e-01-1.52854474e-01j
  -3.41207409e-01-2.47062631e-01j  1.36744145e-01+6.52332220e-02j
   3.28686844e-01+5.54623286e-02j -1.17388285e-15-1.82258580e-16j
   1.62408936e-01-5.62350950e-02j  8.40477299e-03+1.18696805e-01j
   3.4817

SyntaxError: invalid syntax (3746533857.py, line 1)